## Simple Gen AI app using langchain

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
# Data Ingestion
from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader("https://en.wikipedia.org/wiki/24_Hours_of_Le_Mans")
docs=loader.load()
docs

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://en.wikipedia.org/wiki/24_Hours_of_Le_Mans', 'title': '24 Hours of Le Mans - Wikipedia', 'language': 'en'}, page_content='\n\n\n\n24 Hours of Le Mans - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n

In [3]:
# Data transformation
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docus=text_splitter.split_documents(docs)
docus

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/24_Hours_of_Le_Mans', 'title': '24 Hours of Le Mans - Wikipedia', 'language': 'en'}, page_content='24 Hours of Le Mans - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top

In [4]:
# Initializing OpenAI embeddings
from langchain_openai import OpenAIEmbeddings
emb=OpenAIEmbeddings()

In [5]:
# Vector storedb
from langchain_community.vectorstores import FAISS
db=FAISS.from_documents(docus,emb)

In [6]:
# Query from vector storedb
query="The first championship of"
res=db.similarity_search(query)
res[0].page_content

'In 2011, the race became the premier round of the Intercontinental Le Mans Cup, attempting to make a world championship for endurance racing again. In 2012, the race became the centerpiece of the FIA World Endurance Championship, the successor to the ILMC. The 2012 event was the first time the race was won by a hybrid electric vehicle, which was the Audi R18 e-tron quattro.'

In [7]:
# Import LLM
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000212DEF7F8C0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000213012D7B30>, root_client=<openai.OpenAI object at 0x00000212DE104A40>, root_async_client=<openai.AsyncOpenAI object at 0x00000212DEF7F470>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [8]:
## Retriever chain and Document chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following based only on context provided: 
<context>
{context}
</context>
    """
    )

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following based only on context provided: \n<context>\n{context}\n</context>\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000212DEF7F8C0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000213012D7B30>, root_client=<openai.OpenAI object at 0x00000212DE104A40>, root_async_client=<openai.AsyncOpenAI object at 0x00000212DEF7F470>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chai

In [9]:
# Use the chain
from langchain_core.documents import Document
document_chain.invoke({
    "input":"Who is the most successful marque in the history of the race is and their victories?",
    "context":[Document(page_content="The most successful marque in the history of the race is porche")]
})

'The most successful marque in the history of the race is Porsche.'

In [10]:
ret=db.as_retriever()
from langchain.chains import create_retrieval_chain
ret_chain=create_retrieval_chain(ret,document_chain)
ret_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000212DEF7E300>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following based only on context provided: \n<context>\n{context}\n</context>\n    '), additional_kwargs={})])
            | ChatOpenAI(c

In [11]:
# Get response from the LLM
response=ret_chain.invoke({"input":"What is the most successful marque in the history of the race is and their victories?"})
response['answer']

'What is the record speed set during qualifying for the 1988 race, and which car achieved it?\n\nThe record speed set during qualifying for the 1988 race was 407 km/h (253 mph), achieved by a WM Peugeot.'

In [12]:
response

{'input': 'What is the most successful marque in the history of the race is and their victories?',
 'context': [Document(metadata={'source': 'https://en.wikipedia.org/wiki/24_Hours_of_Le_Mans', 'title': '24 Hours of Le Mans - Wikipedia', 'language': 'en'}, page_content="Over the years, many manufacturers have managed to take the overall win, while even more have taken class wins. The most successful marque in the history of the race is Porsche, which has taken nineteen overall victories, including seven in a row from 1981 to 1987 and 107 class victories. Audi is next with thirteen wins,[53][54] and Ferrari follows with eleven, also including six in a row from 1960 to 1965. Since 2000 Audi has dominated the event, winning 13 times in 15 years of participation.[55] Audi and Team Joest have had two hat-tricks, the first being in 2000, 2001, and 2002. Jaguar has seven wins. In contrast, Bentley, Alfa Romeo, and Ford all won four races in a row, with Bentley recording two additional victori